- We loose structure of dialogue using the tokenizer in preprocess_data

- Emotions encoded using MultiLabelBinarizer doesn't tell us
  anymore how many times a single emotion is present in the dialogue
  and where it is

In [1]:
#!pip install torch==1.13.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install transformers==4.30.0
!pip install datasets==2.13.2
!pip install accelerate -U
!pip install evaluate

  Obtaining dependency information for transformers==4.30.0 from https://files.pythonhosted.org/packages/e2/72/1af3d38e98fdcceb3876de4567ac395a66c26976e259fe2d46266e052d61/transformers-4.30.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 71.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.2
    Uninstalling transformers-4.36.2:
      Successfully uninstalled transformers-4.36.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c

In [2]:
# system packages
from pathlib import Path
import shutil
import urllib
import tarfile
import sys
import os
# data and numerical management packages
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
# useful during debugging (progress bars)
from tqdm import tqdm
from transformers import set_seed

seed = 852

random.seed(seed)
np.random.seed(seed)
set_seed(seed)

In [3]:
dataset_folder = Path.cwd().joinpath("MELD_train_efr.json")
#dataset_path = dataset_folder.joinpath('/MELD_train_efr.json')
dataset_folder = "/kaggle/input/plaplapla/MELD_train_efr.json"
df = pd.read_json(dataset_folder)
#df['triggers'] = df['triggers'].fillna(value=0, inplace=False)#.replace('None', 0.0)

In [4]:
df

,episode,speakers,emotions,utterances,triggers
0,utterance_0,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise]",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 1.0, 0.0]"
1,utterance_1,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
2,utterance_2,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
3,utterance_3,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,utterance_4,"[Joey, Rachel, Joey, Rachel]","[surprise, sadness, surprise, fear]",[But then who? The waitress I went out with la...,"[0.0, 0.0, 1.0, 0.0]"
...,...,...,...,...,...
3995,utterance_3995,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3996,utterance_3996,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3997,utterance_3997,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3998,utterance_3998,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [5]:
triggers = df['triggers']
for row in range(triggers.shape[0]):
    for trigger in range(len(triggers[row])):
        if triggers[row][trigger] == None:
            triggers[row][trigger] = 0.0

df['triggers'] = triggers

In [6]:
emotions = df['emotions'].explode().unique()
emotions

array(['neutral', 'surprise', 'fear', 'sadness', 'joy', 'disgust',
       'anger'], dtype=object)

In [7]:
triggers = df['triggers'].explode().unique()
triggers

array([0.0, 1.0], dtype=object)

In [8]:
dialogues = df['utterances']
#print(sentences)
max_len_dialogue = 0
index = 0
for idx, dialogue in enumerate(dialogues):
  if len(dialogue) > max_len_dialogue:
    max_len_dialogue = len(dialogue)
    index = idx
max_len_dialogue,index

(24, 219)

In [9]:
from sklearn.preprocessing import LabelBinarizer
sorted_emotions = sorted(emotions)  #sort the array because Binarizer will automatically do that for one hot encoding
label_binarizer = LabelBinarizer()
label_binarizer.fit(sorted_emotions)

dialogues = df['emotions']
one_hot_emotions = []
for dialogue_emotion in dialogues:
  dialogue_emotions_list = []
  for emotion in dialogue_emotion:
    encoded_emotion=label_binarizer.transform([emotion])
    dialogue_emotions_list.append(np.ravel(encoded_emotion).tolist())
  one_hot_emotions.append(dialogue_emotions_list)

In [10]:
df['emotions'] = one_hot_emotions

In [56]:
from sklearn.model_selection import train_test_split
train_data, temp_data = train_test_split(df, train_size=0.8, shuffle=False)
val_data, test_data = train_test_split(temp_data, test_size=0.5, shuffle=False)
val_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [13]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, AdamW, Trainer, TrainingArguments

class CustomDataset(Dataset):
    def __init__(self, dialogues, emotions, triggers, tokenizer, max_length=10):
        self.dialogues = dialogues
        self.emotions = emotions
        self.triggers = triggers
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
        dialogue = self.dialogues[idx]
        emotion = self.emotions[idx]
        trigger = self.triggers[idx]

        input_ids_list = []
        attention_mask_list = []

        for utterance in dialogue:
          tokenized_utterance = self.tokenizer(utterance, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
          # Extract relevant information
          #input_ids = torch.stack([inputs['input_ids'].squeeze() for inputs in tokenized_dialogue])
          input_ids_list.extend(tokenized_utterance['input_ids'])
          attention_mask_list.extend(tokenized_utterance['attention_mask'])

        emotion_labels = torch.tensor(emotion, dtype=torch.float32)
        trigger_label = torch.tensor(trigger, dtype=torch.long)
        #print('input',torch.stack(input_ids_list).shape)
        #print('attention',torch.stack(attention_mask_list).shape)
        #print('emotion',emotion_labels.shape)
        #print('trigeeer',trigger_label.shape)
        return {
            'input_ids': torch.stack(input_ids_list),
            'attention_mask': torch.stack(attention_mask_list),
            'emotion_labels': emotion_labels,
            'trigger_label': trigger_label
        }

In [107]:
class CustomBERTModel(torch.nn.Module):
    def __init__(self, freeze_embeddings=True):
        super(CustomBERTModel, self).__init__()
        # Replace this with your custom BERT model architecture for multihead classification
        self.bert = BertModel.from_pretrained('bert-base-uncased')#BertForSequenceClassification.from_pretrained
        #LSTM
        if freeze_embeddings:
            for name,param in self.bert.named_parameters():
                if 'embeddings' in name:
                    param.requires_grad = False

        self.emotion_head = torch.nn.Linear(self.bert.config.hidden_size, len(emotions))
        self.trigger_head = torch.nn.Linear(self.bert.config.hidden_size, len(triggers))

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        pooled_output = outputs['pooler_output']

        # Emotion head
        emotion_logits = self.emotion_head(pooled_output)

        # Trigger head
        trigger_logits = self.trigger_head(pooled_output)
        return emotion_logits, trigger_logits
        #return emotion_outputs, trigger_outputs
        #return torch.stack(emotion_outputs), torch.stack(trigger_outputs)

In [108]:
# Assuming train_dialogues, train_emotions, train_triggers, test_dialogues, test_emotions, test_triggers are defined
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_dataset = CustomDataset(train_data['utterances'], train_data['emotions'], train_data['triggers'], tokenizer)
test_dataset = CustomDataset(val_data['utterances'], val_data['emotions'], val_data['triggers'], tokenizer)

freezed_embeddings = True
custom_Bert_Model = CustomBERTModel(freezed_embeddings)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_Bert_Model = custom_Bert_Model.to(device)
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [110]:
num_epochs = 5
batch_size = 1

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    
for epoch in range(num_epochs):
    custom_Bert_Model.train()
    total_loss = 0.0

    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}', leave=False):
        input_ids = batch['input_ids'].squeeze().to(device)
        attention_mask = batch['attention_mask'].squeeze().to(device)
        emotion_labels = batch['emotion_labels'].squeeze().to(device)
        trigger_label = batch['trigger_label'].squeeze().to(device)
        optimizer.zero_grad()
        
        emotion_logits, trigger_logits = custom_Bert_Model(input_ids, attention_mask)

        # Assuming you have defined loss functions for emotion and trigger
        emotion_loss = compute_metrics((emotion_logits, emotion_labels))
        #trigger_loss = your_trigger_loss_function(trigger_logits, trigger_label)

        loss = emotion_loss #+ trigger_loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Loss: {average_loss}")


Epoch 1, Average Loss: 0.03521169826708501


Epoch 2, Average Loss: 0.03519867061200785


Epoch 5, Average Loss: 0.0352083815788501


In [111]:
neutral = (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
surprise =(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)
fear = (0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)
sadness = (0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)
joy = (0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)
disgust = (0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)
anger = (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)
misclassified = (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

emotions_dictionary = {neutral : 'neutral',
                       surprise : 'surprise',
                       fear : 'fear',
                       sadness : 'sadness',
                       joy : 'joy',
                       disgust : 'disgust',
                       anger : 'anger',
                       misclassified : 'misclassified'
}

In [113]:
from sklearn.metrics import f1_score

from sklearn.metrics import f1_score

def compute_sequence_f1(predictions, labels):
    # predictions and labels should be lists of tensors for each dialogue
    emotion_f1_scores = []
    trigger_f1_scores = []
    for emotion_pred, trigger_pred, emotion_lab, trigger_lab in zip(predictions[0], predictions[1], labels[0], labels[1]):
        emotion_predicted_classes = torch.argmax(emotion_pred, dim=1)
        trigger_predicted_classes = torch.argmax(trigger_pred, dim=1)
        emotion_true_classes = torch.argmax(emotion_lab, dim=1)
        trigger_true_classes = trigger_lab
        emotion_f1 = f1_score(emotion_true_classes.cpu().numpy(), emotion_predicted_classes.cpu().numpy(), average='macro')
        trigger_f1 = f1_score(trigger_true_classes.cpu().numpy(), trigger_predicted_classes.cpu().numpy(), average='macro')
        emotion_f1_scores.append(emotion_f1)
        trigger_f1_scores.append(trigger_f1)
    average_emotion_f1 = torch.tensor(emotion_f1_scores, dtype=torch.float32).mean()
    average_trigger_f1 = torch.tensor(trigger_f1_scores, dtype=torch.float32).mean()
    return average_emotion_f1, average_trigger_f1

def compute_unrolled_sequence_f1(predictions, labels):
    # Flatten all utterances and compute the F1 score
    all_emotion_predicted_classes = torch.argmax(torch.cat(predictions[0], dim=0), dim=1)
    all_trigger_predicted_classes = torch.argmax(torch.cat(predictions[1], dim=0), dim=1)
    all_emotion_true_classes = torch.argmax(torch.cat(labels[0], dim=0), dim=1)
    all_trigger_true_classes = torch.cat(labels[1], dim=0)
    unrolled_emotion_f1 = f1_score(all_emotion_true_classes.cpu().numpy(), all_emotion_predicted_classes.cpu().numpy(), average='macro')
    unrolled_trigger_f1 = f1_score(all_trigger_true_classes.cpu().numpy(), all_trigger_predicted_classes.cpu().numpy(), average='macro')
    unrolled_emotion_f1_tensor = torch.tensor(unrolled_emotion_f1, dtype=torch.float32)
    unrolled_trigger_f1_tensor = torch.tensor(unrolled_trigger_f1, dtype=torch.float32)
    return unrolled_emotion_f1_tensor, unrolled_trigger_f1_tensor

# Usage in the eval loop
sequence_f1_scores_emotion = []
sequence_f1_scores_trigger = []
unrolled_predictions_emotion = []
unrolled_predictions_trigger = []
unrolled_labels_emotion = []
unrolled_labels_trigger = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Evaluation', leave=False):
        input_ids = batch['input_ids'].squeeze().to(device)
        attention_mask = batch['attention_mask'].squeeze().to(device)
        emotion_labels = batch['emotion_labels'].squeeze().to(device)
        trigger_label = batch['trigger_label'].squeeze().to(device)

        emotion_logits, trigger_logits = custom_Bert_Model(input_ids, attention_mask)

        # Store predictions and labels for later unrolled F1 computation
        unrolled_predictions_emotion.append(emotion_logits)
        unrolled_labels_emotion.append(emotion_labels)
        unrolled_predictions_trigger.append(trigger_logits)
        unrolled_labels_trigger.append(trigger_label)
        
        # Convert logits to probabilities and then to class predictions
        predicted_classes = torch.argmax(emotion_logits, dim=1)
        true_classes = torch.argmax(emotion_labels, dim=1)

        # Compute F1 for the current sequence (dialogue)
        sequence_f1 = f1_score(true_classes.cpu().numpy(), predicted_classes.cpu().numpy(), average='macro')
        sequence_f1_scores.append(sequence_f1)

# Compute the average Sequence F1 for emotions and triggers
average_sequence_f1_emotion, average_sequence_f1_trigger = compute_sequence_f1(
    [unrolled_predictions_emotion, unrolled_predictions_trigger], 
    [unrolled_labels_emotion, unrolled_labels_trigger]
)

# Compute the Unrolled Sequence F1 for emotions and triggers
unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger = compute_unrolled_sequence_f1(
    [unrolled_predictions_emotion, unrolled_predictions_trigger], 
    [unrolled_labels_emotion, unrolled_labels_trigger]
)

# Print the F1 scores for emotions and triggers
print(f"Average Sequence F1 (Emotion): {average_sequence_f1_emotion}")
print(f"Average Sequence F1 (Trigger): {average_sequence_f1_trigger}")
print(f"Unrolled Sequence F1 (Emotion): {unrolled_sequence_f1_emotion.item()}")
print(f"Unrolled Sequence F1 (Trigger): {unrolled_sequence_f1_trigger.item()}")

Average Sequence F1 (Emotion): 0.04618513211607933
Average Sequence F1 (Trigger): 0.20829695463180542
Unrolled Sequence F1 (Emotion): 0.02938121184706688
Unrolled Sequence F1 (Trigger): 0.1943325698375702
